In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf

data = pd.read_csv('data/vi_processed.csv')

correct_texts = []
error_texts = []

for index, row in data.iterrows():
    if len(correct_texts) == 100: break
    correct_texts.append(row.correct_text)
    error_texts.append(row.error_text)

correct_texts = correct_texts[:1000]
error_texts = error_texts[:1000]


2024-03-31 18:15:48.481305: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-31 18:15:48.486062: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-31 18:15:48.574686: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-31 18:15:48.576715: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-31 18:15:49.533050: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

In [2]:
from tensorflow.keras.preprocessing.text import Tokenizer

vocab_size = 100000
word_level_tokenizer = Tokenizer(num_words=vocab_size, oov_token='<UNK>', lower=True, split=' ', )

word_unk_level_tokenizer = Tokenizer(oov_token='<UNK>', lower=True, split=' ', )

character_vocab_size = 10000
character_level_tokenizer = Tokenizer(num_words=character_vocab_size, lower=True, char_level=True)

In [3]:
word_level_tokenizer.fit_on_texts(correct_texts)
word_level_tokenizer.word_counts

OrderedDict([('nếu', 3),
             ('làm', 22),
             ('được', 18),
             ('như', 10),
             ('vậy', 3),
             ('thì', 5),
             ('chắc', 1),
             ('chắn', 1),
             ('sẽ', 12),
             ('không', 31),
             ('còn', 13),
             ('trường', 5),
             ('nào', 4),
             ('tùy', 1),
             ('tiện', 3),
             ('thu', 1),
             ('tiền', 6),
             ('cao', 4),
             ('gây', 3),
             ('sự', 15),
             ('lo', 2),
             ('lắng', 1),
             ('của', 43),
             ('phụ', 7),
             ('huynh', 1),
             ('và', 33),
             ('ai', 1),
             ('có', 35),
             ('cần', 4),
             ('đóng', 2),
             ('mọi', 2),
             ('liên', 2),
             ('lạc', 1),
             ('báo', 6),
             ('cáo', 3),
             ('phê', 1),
             ('duyệt', 2),
             ('hướng', 1),
             ('dẫn', 3),
  

In [4]:
character_level_tokenizer.fit_on_texts(error_texts)
character_level_tokenizer.word_counts

OrderedDict([('n', 1237),
             ('c', 618),
             ('u', 295),
             ('l', 184),
             ('à', 204),
             ('m', 253),
             ('y', 162),
             ('s', 171),
             ('đ', 256),
             ('ư', 133),
             ('ợ', 32),
             (' ', 2765),
             ('h', 955),
             ('v', 192),
             ('ậ', 39),
             ('t', 697),
             ('ì', 51),
             ('ắ', 35),
             ('e', 71),
             ('k', 152),
             ('ô', 82),
             ('f', 15),
             ('g', 565),
             ('ò', 27),
             ('r', 192),
             ('b', 126),
             ('ờ', 49),
             ('o', 252),
             ('ù', 21),
             ('i', 602),
             ('ệ', 70),
             ('ề', 54),
             ('a', 350),
             ('â', 51),
             ('ự', 44),
             ('ủ', 63),
             ('p', 169),
             ('ụ', 25),
             ('ó', 90),
             ('ầ', 39),
             ('ọ

In [5]:
word_unk_level_tokenizer.fit_on_texts(error_texts)
word_unk_level_tokenizer.word_counts

OrderedDict([('nculàmysđược', 1),
             ('như', 9),
             ('vậy', 3),
             ('thì', 4),
             ('chắc', 1),
             ('chắn', 1),
             ('se', 2),
             ('khôfng', 1),
             ('còn', 10),
             ('trbường', 1),
             ('nào', 4),
             ('tùy', 1),
             ('tiện', 3),
             ('thu', 2),
             ('tiền', 6),
             ('cao', 4),
             ('gây', 3),
             ('sự', 11),
             ('lo', 2),
             ('lắng', 1),
             ('của', 36),
             ('phụ', 6),
             ('huynh', 1),
             ('và', 32),
             ('ai', 1),
             ('không', 26),
             ('có', 27),
             ('cần', 4),
             ('đóng', 2),
             ('mọi', 2),
             ('liên', 2),
             ('lạc', 1),
             ('báo', 5),
             ('cáo', 2),
             ('phê', 1),
             ('duyệt', 2),
             ('hướng', 1),
             ('dẫn', 3),
             ('giữa

In [6]:
MAX_WORD_LEVEL_TOKENS = 128
MAX_CHARACTER_LEVEL_TOKENS = 200

In [7]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

input_sequences = word_level_tokenizer.texts_to_sequences(error_texts)
output_sequences = word_level_tokenizer.texts_to_sequences(correct_texts)
unk_input_sequences = word_unk_level_tokenizer.texts_to_sequences(error_texts)

character_level_input_sequences = []

for sequence in unk_input_sequences:
    character_level_input_sequence = []
    for word_token in sequence:
        word = word_unk_level_tokenizer.index_word[word_token]
        word = character_level_tokenizer.texts_to_sequences(word)
        word_chars = [each[0] for each in word]
        character_level_input_sequence.append(word_chars)
    character_level_input_sequence = pad_sequences(character_level_input_sequence, maxlen=MAX_CHARACTER_LEVEL_TOKENS, padding='post', truncating='post')
    character_level_input_sequences.append(character_level_input_sequence)



In [8]:



def prepare_batch(input, output):
    word_level_input = word_level_tokenizer.texts_to_sequences(input)
    word_level_input = tf.ragged.constant(word_level_input)
    word_level_input = word_level_input[:, :MAX_WORD_LEVEL_TOKENS]
    word_level_input = word_level_input.to_tensor()

    unk_input_sequences = word_unk_level_tokenizer.texts_to_sequences(input)
    character_level_input_sequences = []

    for sequence in unk_input_sequences:
        character_level_input_sequence = []
        for word_token in sequence:
            word = word_unk_level_tokenizer.index_word[word_token]
            word = character_level_tokenizer.texts_to_sequences(word)
            word_chars = [each[0] for each in word]
            character_level_input_sequence.append(word_chars)
        character_level_input_sequence = tf.ragged.constant(character_level_input_sequence)
        character_level_input_sequence = character_level_input_sequence[:MAX_WORD_LEVEL_TOKENS,
                                         :MAX_CHARACTER_LEVEL_TOKENS]
        character_level_input_sequence = character_level_input_sequence.to_tensor()
        character_level_input_sequences.append(character_level_input_sequence)

    print(character_level_input_sequences)
    character_level_input_sequences = tf.ragged.constant(character_level_input_sequences)

    output = word_level_tokenizer.texts_to_sequences(output)
    output = tf.ragged.constant(output)
    output = output[:, :MAX_WORD_LEVEL_TOKENS]
    output = output.to_tensor()

    return (word_level_input, character_level_input_sequences), output

In [9]:
# print(prepare_batch(['nhanh di hoc day', 'xin day'], ['xin nam', 'xin']))

In [10]:
BUFFER_SIZE = 20000
BATCH_SIZE = 64

In [11]:
from HierarchicalTransformerEncoder import HierarchicalTransformerEncoder

model = HierarchicalTransformerEncoder(num_character_level_layers=4,
                                       num_word_level_layers=12,
                                       character_level_d_model=128,
                                       word_level_d_model=786,
                                       num_heads=3, dff=512,
                                       vocab_size=100000, character_vocab_size=10000)

In [12]:
from HierarchicalTransformerEncoder import custom_loss

model.compile(optimizer='adam', loss=custom_loss)

In [13]:
input_sequences = pad_sequences(input_sequences, maxlen=MAX_WORD_LEVEL_TOKENS, padding='post', truncating='post')
output_sequences = pad_sequences(output_sequences, maxlen=MAX_WORD_LEVEL_TOKENS, padding='post', truncating='post')

character_level_input_sequences = pad_sequences(character_level_input_sequences, maxlen=MAX_WORD_LEVEL_TOKENS, padding='post',
                                                 truncating='post')

input_sequences_np = np.array(input_sequences)
character_level_input_sequences_np = np.array(character_level_input_sequences)
output_sequences_np = np.array(output_sequences)

In [14]:
output = model(input_sequences_np, character_level_input_sequences_np)

tf.Tensor(
[[  1  41 223 ...   0   0   0]
 [332 333 545 ...   0   0   0]
 [235 125 555 ...   0   0   0]
 ...
 [ 29  65  92 ...   0   0   0]
 [396 536 535 ...   0   0   0]
 [  6  57 484 ...   0   0   0]], shape=(100, 128), dtype=int32)
[[[ 2  5  9 ...  0  0  0]
  [ 2  3 22 ...  0  0  0]
  [14 48 19 ...  0  0  0]
  ...
  [ 0  0  0 ...  0  0  0]
  [ 0  0  0 ...  0  0  0]
  [ 0  0  0 ...  0  0  0]]

 [[11 47  6 ...  0  0  0]
  [16  6 33 ...  0  0  0]
  [16 26  5 ...  0  0  0]
  ...
  [ 0  0  0 ...  0  0  0]
  [ 0  0  0 ...  0  0  0]
  [ 0  0  0 ...  0  0  0]]

 [[23 13  2 ...  0  0  0]
  [ 4  3 52 ...  0  0  0]
  [24  9 19 ...  0  0  0]
  ...
  [ 0  0  0 ...  0  0  0]
  [ 0  0  0 ...  0  0  0]
  [ 0  0  0 ...  0  0  0]]

 ...

 [[ 5  3 71 ...  0  0  0]
  [ 2 33  2 ...  0  0  0]
  [ 4  6 29 ...  0  0  0]
  ...
  [ 0  0  0 ...  0  0  0]
  [ 0  0  0 ...  0  0  0]
  [ 0  0  0 ...  0  0  0]]

 [[10 22 46 ...  0  0  0]
  [20  6 11 ...  0  0  0]
  [ 3 16 14 ...  0  0  0]
  ...
  [ 0  0  0 ...  0 

2024-03-31 18:16:05.708050: W tensorflow/tsl/framework/bfc_allocator.cc:485] Allocator (mklcpu) ran out of memory trying to allocate 156.25GiB (rounded to 167772160000)requested by op ResourceGather
If the cause is memory fragmentation maybe the environment variable 'TF_GPU_ALLOCATOR=cuda_malloc_async' will improve the situation. 
Current allocation summary follows.
Current allocation summary follows.
2024-03-31 18:16:05.708091: I tensorflow/tsl/framework/bfc_allocator.cc:1039] BFCAllocator dump for mklcpu
2024-03-31 18:16:05.708107: I tensorflow/tsl/framework/bfc_allocator.cc:1046] Bin (256): 	Total Chunks: 0, Chunks in use: 0. 0B allocated for chunks. 0B in use in bin. 0B client-requested in use in bin.
2024-03-31 18:16:05.708115: I tensorflow/tsl/framework/bfc_allocator.cc:1046] Bin (512): 	Total Chunks: 0, Chunks in use: 0. 0B allocated for chunks. 0B in use in bin. 0B client-requested in use in bin.
2024-03-31 18:16:05.708122: I tensorflow/tsl/framework/bfc_allocator.cc:1046] Bin 

ResourceExhaustedError: Exception encountered when calling layer 'embedding_2' (type Embedding).

{{function_node __wrapped__ResourceGather_device_/job:localhost/replica:0/task:0/device:CPU:0}} OOM when allocating tensor with shape[100,128,200,128,128] and type float on /job:localhost/replica:0/task:0/device:CPU:0 by allocator mklcpu [Op:ResourceGather] name: 

Call arguments received by layer 'embedding_2' (type Embedding):
  • inputs=tf.Tensor(shape=(100, 128, 200, 128), dtype=float32)

In [ ]:



model.fit([input_sequences_np, character_level_input_sequences_np], output_sequences_np, epochs=10, batch_size=20)

In [ ]:
character_level_input_sequences